# OpenET DisALEXI
## Image Mapping Examples
This example shows how you could build an ET image collection by mapping the model image class over a GEE Landsat image collection.

In [ ]:
import pprint

import ee

from IPython.display import Image
import openet.disalexi as model

In [ ]:
ee.Initialize()

### Input parameters

In [ ]:
collection_id = 'LANDSAT/LC08/C02/T1_L2'

# Date range you want to aggregate ET over
# End date is inclusive
start_date = '2017-07-01'
end_date = '2017-08-01'

# Hard code the study area and CRS to the test point location
# The region must be quite small in order to not have GEE memory errors or time outs
test_xy = [-121.5265, 38.7399]
test_point = ee.Geometry.Point(test_xy)
study_area = ee.Geometry.Rectangle(test_xy[0] - 0.2, test_xy[1] - 0.1, test_xy[0] + 0.2, test_xy[1] + 0.1)
region = study_area.bounds(1, 'EPSG:4326')
crs = 'EPSG:32610'

In [ ]:
image_size = 768
landsat_cs = 30

In [ ]:
ndvi_palette = ['#EFE7E1', '#003300']
et_palette = [
    'DEC29B', 'E6CDA1', 'EDD9A6', 'F5E4A9', 'FFF4AD', 'C3E683', '6BCC5C', 
    '3BB369', '20998F', '1C8691', '16678A', '114982', '0B2C7A']

### Landsat 8 Collection 2 SR Images
Build the input image collection

In [ ]:
landsat_coll = (
    ee.ImageCollection(collection_id)
    .filterDate(start_date, end_date)
    .filterBounds(test_point)
)

# Check which images are in the collection
pprint.pprint(list(landsat_coll.aggregate_histogram('system:index').getInfo().keys()))

In [ ]:
image_url = (
    landsat_coll.first().select(['SR_B4', 'SR_B3', 'SR_B2'])
    .multiply(0.0000275).add(-0.2)
    .getThumbURL({'min': 0.0, 'max': 0.3, 'gamma': 1.25, 'region': region, 'dimensions': image_size})
)
Image(image_url, embed=True, format='png')

In [ ]:
landsat_cs = 30
landsat_crs = ee.Image(landsat_coll.first()).select('SR_B3').projection().getInfo()['crs']
landsat_region = ee.Image(landsat_coll.first()).geometry().bounds(1, 'EPSG:4326').coordinates().getInfo()
# landsat_dt = ee.Date(landsat_img.get('system:time_start'))

### ET Image Collection

In [ ]:
def compute_et(image):
    """Return an ET image for each input Landsat 8 C2 SR image"""
    return model.Image.from_landsat_c02_l2(
        image, 
        lai_source='projects/openet/assets/lai/landsat/c02'
    ).et
    
# Build the DisALEXI model for each image then compute and return ET
et_coll = ee.ImageCollection(landsat_coll.map(compute_et))

In [ ]:
# Note, a thumbnail can't be generated for a full Landsat image at scale=30
image_url = (
    ee.Image(et_coll.mean())
    .getThumbURL({'min': 0, 'max': 12, 'palette': et_palette, 'region': region, 'dimensions': image_size})
)
Image(url=image_url, embed=True, format='png')

### Custom Image Class Parameters 
It is also possible to pass custom parameter to the Image class as long as they are the same for all images.

In [ ]:
def custom_et(image):
    image_obj = model.Image.from_landsat_c02_l2(
        image, 
        lai_source='projects/openet/assets/lai/landsat/c02',
        stabil_iterations=2,
    )
    return image_obj.et
    
# Build the DisALEXI model for each image then compute and return ET
et_coll = ee.ImageCollection(landsat_coll.map(custom_et))

In [ ]:
image_url = (
    ee.Image(et_coll.mean())
    .getThumbURL({'min': 0, 'max': 12, 'palette': et_palette, 'region': region, 'dimensions': image_size})
)
Image(url=image_url, embed=True, format='png')

### Map the calculate method over the Landsat collection

In [ ]:
def compute_vars(image):
    """Return an ET image for each input Landsat 8 C2 SR image"""
    return model.Image.from_landsat_c02_l2(
        image, 
        lai_source='projects/openet/assets/lai/landsat/c02'
    ).calculate(['et'])
    
# Build the DisALEXI model for each image then compute and return ET
vars_coll = ee.ImageCollection(landsat_coll.map(compute_vars))

In [ ]:
image_url = (
    ee.Image(vars_coll.select(['et']).mean())
    .getThumbURL({'min': 0, 'max': 12, 'palette': et_palette, 'region': region, 'dimensions': image_size})
)
Image(url=image_url, embed=True, format='png')